## Missing data

- We have different reasons for NA values, error in data collection, measurements not applicable, spaces left blank.

### Identifyng missing values in tabular data

In [17]:
import pandas as pd
from io import StringIO

csv_data = \
'''A,B,C,D
1.0,2.0,3.0,4.0
5.0,6.0,,8.0
10.0,11.0,12.0,'''

df = pd.read_csv(StringIO(csv_data))
df

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,NaN,8.0
2,10.0,11.0,12.0,NaN


- We can use the `isnull` method to return a Dataframe with boolean values that indicates if each column have missing values or not.

In [2]:
df.isnull().sum()

A    0
B    0
C    1
D    1
dtype: int64

In [3]:
### Note
### Scikit learn was originally developed to work with Numpy arrays
### Sometimes is better to work with the Numpy arrays rather than with the DataFrames
df.values

array([[ 1.,  2.,  3.,  4.],
       [ 5.,  6., nan,  8.],
       [10., 11., 12., nan]])

### Eliminating training examples or features with missing values

- The easiest way is to remove whole columns or rows.

In [4]:
df.dropna(axis=0)

,A,B,C,D
0,1.0,2.0,3.0,4.0


In [5]:
df.dropna(axis=1)

,A,B
0,1.0,2.0
1,5.0,6.0
2,10.0,11.0


In [6]:
# only drop rows where all columns are NA
df.dropna(how="all")

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,NaN,8.0
2,10.0,11.0,12.0,NaN


In [7]:
# drop rows that have fewer than 4 real values
df.dropna(thresh=4)

,A,B,C,D
0,1.0,2.0,3.0,4.0


In [8]:
# only drop rows where Na appear in specific column
df.dropna(subset=["C"])

,A,B,C,D
0,1.0,2.0,3.0,4.0
2,10.0,11.0,12.0,NaN


- Several disadvantages for these methods.

### Imputing missing values

- Estimate the missing values from the other training examples.
- One of the most common techniques is **mean imputation**.

In [18]:
from sklearn.impute import SimpleImputer
import numpy as np
imr = SimpleImputer(missing_values=np.nan, strategy="mean")
imr = imr.fit(df.values)
imputed_data = imr.transform(df.values)
imputed_data

array([[ 1. ,  2. ,  3. ,  4. ],
       [ 5. ,  6. ,  7.5,  8. ],
       [10. , 11. , 12. ,  6. ]])

In [19]:
# we can also use pandas
df.fillna(df.mean())

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,7.5,8.0
2,10.0,11.0,12.0,6.0


### Scikit learn estimator API

- Two essential methods: fit and transform.
- fit is used to learn the parameters from the training data.
- transform uses those parameters to transform the data.

### Handling categorical data

- We have **ordinal** and **nominal** features.

#### Categorical data encoding with pandas


In [20]:
df = pd.DataFrame([
    ["green", "M", 10.1, "class2"],
    ["red", "L", 13.5, "class1"],
    ["blue", "XL", 15.3, "class2"]])
df.columns = ["color", "size", "price", "classlabel"]
df

,color,size,price,classlabel
0,green,M,10.1,class2
1,red,L,13.5,class1
2,blue,XL,15.3,class2


### Mapping ordinal features

- We have to automatically define the mapping function.

In [21]:
size_mapping = {"XL": 3,
                "L": 2,
                "M": 1}

df["size"] = df["size"].map(size_mapping)
df

,color,size,price,classlabel
0,green,1,10.1,class2
1,red,2,13.5,class1
2,blue,3,15.3,class2


In [22]:
# if we want to inverse it
inv_size_mapping = {v: k for k, v in size_mapping.items()}
df["size"].map(inv_size_mapping)

0     M
1     L
2    XL
Name: size, dtype: object

### Encoding class labels

- class labels are not ordinal.
- Good practice to do the encoding manually to avoid bugs.

In [23]:
class_mapping = {label: idx for idx, label in
                 enumerate(np.unique(df["classlabel"]))}
class_mapping

{'class1': 0, 'class2': 1}

In [24]:
# now we use the mapping dictionary to transform the class labels
df["classlabel"] = df["classlabel"].map(class_mapping)
df

,color,size,price,classlabel
0,green,1,10.1,1
1,red,2,13.5,0
2,blue,3,15.3,1


In [25]:
# we can also reverse it
inv_class_mapping = {v: k for k, v in class_mapping.items()}
df["classlabel"] = df["classlabel"].map(inv_class_mapping)
df

,color,size,price,classlabel
0,green,1,10.1,class2
1,red,2,13.5,class1
2,blue,3,15.3,class2


In [26]:
# There is a label encoder in scikit learn
from sklearn.preprocessing import LabelEncoder
class_le = LabelEncoder()
y = class_le.fit_transform(df["classlabel"].values)
y

array([1, 0, 1])

In [27]:
class_le.inverse_transform(y)

array(['class2', 'class1', 'class2'], dtype=object)

### One hot encoding on nominal features

- We could use the label encoder to transform the color feature

In [28]:
X = df[["color", "size", "price"]].values
color_le = LabelEncoder()
X[:, 0] = color_le.fit_transform(X[:, 0])
X

array([[1, 1, 10.1],
       [2, 2, 13.5],
       [0, 3, 15.3]], dtype=object)

- We changed the colors into integers 1, 2, 3.
- This is an error, the model might think that green is larger than blue and red is larger than green.
- The workaround is called **one-hot encoding**.
- We create a new dummy feature for each unique value in the nominal feature column.

In [31]:
from sklearn.preprocessing import OneHotEncoder
X = df[["color", "size", "price"]].values
color_ohe = OneHotEncoder()
color_ohe.fit_transform(X[:, 0].reshape(-1, 1)).toarray()

array([[0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.]])

In [33]:
# column trasnformer accepts a list of (name, transformer, columns) tuples
from sklearn.compose import ColumnTransformer
X = df[["color", "size", "price"]].values
c_trasnform = ColumnTransformer([
    ("onehot", OneHotEncoder(), [0]),
    ("nothing", "passthrough", [1, 2])
])
c_trasnform.fit_transform(X).astype(float)

array([[ 0. ,  1. ,  0. ,  1. , 10.1],
       [ 0. ,  0. ,  1. ,  2. , 13.5],
       [ 1. ,  0. ,  0. ,  3. , 15.3]])

In [34]:
# we can also use pandas method
pd.get_dummies(df[["price", "color", "size"]])

,price,size,color_blue,color_green,color_red
0,10.1,1,False,True,False
1,13.5,2,False,False,True
2,15.3,3,True,False,False


- One hot encoding introduces multicollinearity.
- To remove this problem we can remove on feature column from the one hot encoded array.
- We do not lose any important information doing this.

In [35]:
pd.get_dummies(df[["price", "color", "size"]],
               drop_first=True)

,price,size,color_green,color_red
0,10.1,1,True,False
1,13.5,2,False,True
2,15.3,3,False,False


In [37]:
color_ohe = OneHotEncoder(categories="auto", drop="first")
c_transf = ColumnTransformer([
    ("onehot", color_ohe, [0]),
    ("nothing", "passthrough", [1, 2])
])
c_transf.fit_transform(X).astype(float)

array([[ 1. ,  0. ,  1. , 10.1],
       [ 0. ,  1. ,  2. , 13.5],
       [ 0. ,  0. ,  3. , 15.3]])

## Partitioning a dataset into separate training and test datasets

- We will use the wine dataset.

In [41]:
df_wine = pd.read_csv(
    'https://archive.ics.uci.edu/ml/'
    'machine-learning-databases/wine/wine.data',
    header=None
)

In [42]:
df_wine.columns = ['Class label', 'Alcohol',
    'Malic acid', 'Ash',
    'Alcalinity of ash', 'Magnesium',
    'Total phenols', 'Flavanoids',
    'Nonflavanoid phenols',
    'Proanthocyanins',
    'Color intensity', 'Hue',
    'OD280/OD315 of diluted wines',
    'Proline']

In [43]:
print("Class labels", np.unique(df_wine["Class label"]))

Class labels [1 2 3]


In [44]:
df_wine

,Class label,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,OD280/OD315 of diluted wines,Proline
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,3,13.71,5.65,2.45,20.5,95,1.68,0.61,0.52,1.06,7.70,0.64,1.74,740
174,3,13.40,3.91,2.48,23.0,102,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750
175,3,13.27,4.28,2.26,20.0,120,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835
176,3,13.17,2.59,2.37,20.0,120,1.65,0.68,0.53,1.46,9.30,0.60,1.62,840


- We use the train_test_split function

In [45]:
from sklearn.model_selection import train_test_split
X, y = df_wine.iloc[:, 1:].values, df_wine.iloc[:, 0].values
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.3,
                                                    random_state=0,
                                                    stratify=y)

- The test size depends on the size of our dataset, 30, 40, 20% are good splits.
- If we have really large datasets we can consider even 10% or 1%.
